<a href="https://colab.research.google.com/github/amyth18/CS598-Deep-Learning-Final-Project/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
! pip install gensim --upgrade

In [30]:
import pandas as pd

In [31]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [32]:
# read data
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
df.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

## Common data preprocessing
1. Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

2. Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

3. Negative filters (e.g. "no sign of breath problem").

## Data processing for X1 (Word2Vec)
1. Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

## Data processing for X2 (TF-IDF)
1. Generate TF-IDF vector for each symptom entity.

## Data processing for y
1. multi-hot encoding?
 




In [34]:
import re
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          text = self.sanitize_text(self.dframe["TEXT"][idx])
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df_icd_codes = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

In [36]:
df_icd_codes.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486


In [37]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()
df_admissions_with_top_diseases = df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]
df_unique_admissions_with_top_diseases = df_admissions_with_top_diseases[["SUBJECT_ID", "HADM_ID"]].drop_duplicates()

In [38]:
len(df_unique_admissions_with_top_diseases)

55078

In [39]:
filtered_df = pd.merge(df, df_unique_admissions_with_top_diseases, on=["SUBJECT_ID", "HADM_ID"])

In [40]:
filtered_df_disc_sum = filtered_df[filtered_df["CATEGORY"] == 'Discharge summary'].reset_index()
print(len(filtered_df_disc_sum))
print(filtered_df_disc_sum.head(5))

55988
   index  ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0      0     174       22532  167853.0  2151-08-04       NaN       NaN   
1      1     170       22532  167853.0  2151-08-04       NaN       NaN   
2     32     175       13702  107527.0  2118-06-14       NaN       NaN   
3     79     176       13702  167118.0  2119-05-25       NaN       NaN   
4    186     177       13702  196489.0  2124-08-18       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1  Discharge summary      Report   NaN      NaN   
2  Discharge summary      Report   NaN      NaN   
3  Discharge summary      Report   NaN      NaN   
4  Discharge summary      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2151-7-16**]       Dischar...  
1  Admission Date:  [**2151-7-16**]       Dischar...  
2  Admission Date:  [**2118-6-2**]       Discharg...  
3  Admission Date:  [*

Test code for sentence generator

Word2Vec training using gensim.

In [43]:
!ls -lh "/content/drive/My Drive/DLH Final Project/mimic3/"

total 3.9G
-rw------- 1 root root  19M Mar 19  2019 DIAGNOSES_ICD.csv
-rw------- 1 root root 3.8G Jun 14  2019 NOTEEVENTS.csv
-rw------- 1 root root  51M Apr 14 07:48 word2vec-4.model
-rw------- 1 root root  78M Apr 13 08:14 word2vec.model


In [44]:
# multi-hot encoding for icd_codes for a given admission

series_admissions_with_top_diseases_grouped = \
df_admissions_with_top_diseases.groupby(['SUBJECT_ID', 
                                         'HADM_ID'])['ICD9_CODE'].apply(list)

df_admissions_with_top_diseases_grouped = \
series_admissions_with_top_diseases_grouped.to_frame().reset_index()

In [45]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

In [46]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_admissions_with_top_diseases_grouped)):
  icd_codes = df_admissions_with_top_diseases_grouped.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

df_admissions_with_top_diseases_grouped['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

In [48]:
# create our final dataset by joining 
# df_admissions_with_top_diseases_grouped and filtered_df_disc_sum
df_dataset = pd.merge(filtered_df_disc_sum, 
                      df_admissions_with_top_diseases_grouped, 
                      on=["SUBJECT_ID", "HADM_ID"])

In [ ]:
# extract symptoms from the text, currently we treat all tokens as symptoms
# need to add all the filters discussed in the paper.
sgen = MySentences(df_dataset)
symptom_col = list()
for s in sgen:
  symptom_col.append(s)
  
# add the new column to the dataset.
df_dataset["SYMPTOMS"] = symptom_col

# Generate Word2Vec Embeddings

In [42]:
# NOTE: commenting this part so that we dont run this by mistake.

# import gensim
# sgen = MySentences(df_dataset) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model")

# Construct dataset with Word2Vec embeddings

In [51]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model')

In [ ]:
vector = model.wv['hemorrhage']
vector

In [53]:
model.wv.most_similar('hemorrhage', topn=10)

[('intraparenchymal', 0.902775228023529),
 ('subarachnoid', 0.8910377621650696),
 ('intraventricular', 0.8485040664672852),
 ('intracerebral', 0.8260278105735779),
 ('temporoparietal', 0.8176181316375732),
 ('thalamic', 0.8115888833999634),
 ('evolution', 0.8070071339607239),
 ('hemispheric', 0.8048758506774902),
 ('frontoparietal', 0.8016946911811829),
 ('intracranial', 0.7956364750862122)]

In [54]:
X_word2vec = list()
for idx in range(len(df_dataset)):
  # ignore words in not vocabulary
  symptoms = df_dataset["SYMPTOMS"]
  symptoms_emb = [model.wv[s] for s in symptoms if w in model.wv]
  X_word2vec.append(symptoms_emb)

# multi-hot encoding for ICD codes diagnosed.
y = df_dataset['ICD9_CODE_ENCODED']

In [1]:
import pickle
pfile = open("/content/drive/My Drive/DLH Final Project/X_word2vec", "ab")
pickle.dump(X_word2vec, pfile)
pfile.close()

NameError: ignored

# Construct data with TF-IDF encoding

In [65]:
import numpy as np
import itertools

vocab_size = len(model.wv)
tf = np.zeros((len(model.wv), len(top_icd_codes)))
sgen = MySentences(df_dataset)
for idx, s in enumerate(sgen):
  # XXX: TODO currently we treat all tokens from "TEXT" as sypmtoms
  # get the icd codes for this discharge summary
  icd_codes = df_dataset['ICD9_CODE'][idx]
  # create a cross product of symptoms and icd codes
  # and update tf matrix. tf matrix keeps count of how many 
  # (i.e frequency) times <symptom, icd code> pair occur in our dataset.
  for pair in itertools.product(s, icd_codes):
    # update count of each (symptom, icd_code) pair to compute TF
    if pair[0] in model.wv:
      tf[model.wv.get_index(pair[0])][icd_code_to_idx[pair[1]]] += 1

# Complete the TF-IDF matrix computation.
# Compute the number of ICD Codes (i.e diseaes) each 
# symptom is associated with.
D_i = np.sum(tf > 0, axis=1)
print(D_i.shape)

log_N_Di = np.log(len(top_icd_codes)/D_i)
tf_idf = (tf.T * log_N_Di).T
print(tf_idf.shape)

(64259,)
(64259, 50)


In [66]:
import pickle
pfile = open("/content/drive/My Drive/DLH Final Project/tf-idf", "ab")
pickle.dump(tf_idf, pfile)
pfile.close()

In [69]:
# build the X_tfidf dataset
sgen = MySentences(df_dataset)
for idx, s in enumerate(sgen):


total 25M
-rw------- 1 root root  54K Apr 13 17:57 'CS598 Deep Learning For Healthcare Final Project'
drwx------ 2 root root 4.0K Apr 13 18:46  metamap
drwx------ 2 root root 4.0K Apr 14 07:48  mimic3
-rw------- 1 root root  25M Apr 14 10:05  tf-idf
